In [40]:
import pandas as pd
import numpy as np
import datetime
from sklearn.ensemble import RandomForestClassifier

# SELECT train data 
train = pd.read_csv('./train_nt6.csv')

In [41]:
test = pd.read_csv('./input/14p.csv')
test_time = test.sort_values(by=['cur_time'], axis=0)

idx1 = test_time[(test_time['temp_max'] > 50) | (test_time['temp_min'] < -20)].index
test_time = test_time.drop(idx1)
idx2 = test_time[(test_time['hum_max'] > 100) | (test_time['hum_min'] < 0)].index
test_time = test_time.drop(idx2)

num = test_time.shape[0]
test_time = test_time.reindex(range(num), method='ffill')

f = open("test14p.csv", "w")
f.write('temp_avg_gap')
f.write(",")
f.write('hum_avg_gap')
f.write(",")
f.write('temp_max_min')
f.write(",")
f.write('temp_min_max')
f.write(",")
f.write('hum_max_min')
f.write(",")
f.write('hum_min_max')
f.write(",")
f.write('status')
f.write("\n")
for i in range(test_time.shape[0]-1):
    if test_time['status'][i] != 22:
        if test_time['status'][i+1] == test_time['status'][i] and test_time['serial'][i+1] == test_time['serial'][i]:
            data = datetime.datetime.strptime(test_time['cur_time'][i+1], '%Y-%m-%dT%H:%M:%S.%fZ') - datetime.datetime.strptime(test_time['cur_time'][i], '%Y-%m-%dT%H:%M:%S.%fZ')
            res = round(data.seconds + data.days * 24 * 3600)
            if res < 100:
                f.write(str(round(test_time['temp_avg'][i+1] - test_time['temp_avg'][i], 2)))
                f.write(",")
                f.write(str(round(test_time['hum_avg'][i+1] - test_time['hum_avg'][i], 2)))
                f.write(",")
                f.write(str(round(test_time['temp_max'][i+1] - test_time['temp_min'][i], 2)))
                f.write(",")
                f.write(str(round(test_time['temp_min'][i+1] - test_time['temp_max'][i], 2)))
                f.write(",")
                f.write(str(round(test_time['hum_max'][i+1] - test_time['hum_min'][i], 2)))
                f.write(",")
                f.write(str(round(test_time['hum_min'][i+1] - test_time['hum_max'][i], 2)))
                f.write(",")
                f.write(str(test_time['status'][i]))
                f.write("\n")
f.close()

test = pd.read_csv('./test14p.csv')

In [42]:
train_data = train.drop('work', axis=1)
target = train["work"]

In [43]:
clf = RandomForestClassifier(n_estimators=7)
clf.fit(train_data, target)

test_data = test.copy()
predict = clf.predict(test_data)
result =pd.DataFrame({
    'status': test['status'],
    'work': predict
})
result.to_csv('result_RF_14.csv', index=False)
result_RF = pd.read_csv('result_RF_14.csv')

In [44]:
print("RF  count: ", result_RF['work'].value_counts()[1])
print("RF  error: ", result_RF['work'].value_counts()[0])
print("Accuracy: ", (result_RF['work'].value_counts()[1]/(result_RF['work'].value_counts()[1]+result_RF['work'].value_counts()[0])))

RF  count:  93
RF  error:  36
Accuracy:  0.7209302325581395


In [45]:
for i in range(0, result_RF.shape[0]-1, 2):
    if result_RF['status'][i] == result_RF['status'][i + 1]:
        if (result_RF['work'][i] | result_RF['work'][i + 1]) == 0:
            print("ERROR OCCUR in status ", result_RF['status'][i])
            print(result_RF['work'][i])
            print(result_RF['work'][i+1])
            print(i)

ERROR OCCUR in status  23
0
0
18
ERROR OCCUR in status  31
0
0
40
ERROR OCCUR in status  13
0
0
98
ERROR OCCUR in status  13
0
0
102
ERROR OCCUR in status  13
0
0
106
ERROR OCCUR in status  13
0
0
110
ERROR OCCUR in status  13
0
0
112
ERROR OCCUR in status  13
0
0
116
ERROR OCCUR in status  13
0
0
118
ERROR OCCUR in status  13
0
0
120
ERROR OCCUR in status  13
0
0
124
ERROR OCCUR in status  13
0
0
126
